# SEIR-Campus Examples

This file illustrates some of the examples from the corresponding paper on the SEIR-Courses package.  Many of the function here call on classes inside the SEIR-Courses package.  We encourage you to look inside the package and explore!

The data file that comes in the examples, publicdata.data, is based on course network published by Weeden and Cornwell at https://osf.io/6kuet/.  Course names, as well as some student demographic information, and varsity athlete ids have been assigned randomly and are not accurate representations of student demographics and varsity athletics at Cornell University.

In [ ]:
from datetime import datetime, timedelta
from PySeirCampus import *

### Load the data for the simulations.

In [ ]:
holiday_list = [(2020, 10, 14)]
holidays = set(datetime(*h) for h in holiday_list)
semester = Semester('publicdata.data', holidays)

### Run a first simulation!

In [ ]:
parameters = Parameters(reps = 10)
run_repetitions(semester, parameters)

### Example when no one self reports/shows symptoms.

In [ ]:
parameters = Parameters(reps = 10)
parameters.infection_duration = BasicInfectionDuration(1 / 3.5, 1 / 4.5)
run_repetitions(semester, parameters)

### Example of infection testing

In [ ]:
parameters = Parameters(reps = 10)
parameters.intervention_policy = IpWeekdayTesting(semester)
run_repetitions(semester, parameters)

### Example with Contact Tracing and Quarantines

In [ ]:
parameters = Parameters(reps = 10)
parameters.contact_tracing = BasicContactTracing(14)
run_repetitions(semester, parameters)

### Example with hybrid classes

In [ ]:
semester_alt_hybrid = make_alternate_hybrid(semester)
parameters = Parameters(reps = 10)
run_repetitions(semester_alt_hybrid, parameters)

### Building social groups.

For proper randomization, these groups should be recreated for each simulation repetition using the preprocess feature in the parameters.  However, this can add significant time to the computations.

First, consider generating random clusters of students each day.

In [ ]:
settings = ClusterSettings(
        start_date = min(semester.meeting_dates), end_date = max(semester.meeting_dates),
        weekday_group_count = 280, weekday_group_size = 10, weekday_group_time = 120,
        weekend_group_count = 210, weekend_group_size = 20, weekend_group_time = 180)

def groups_random(semester):
    clusters = make_randomized_clusters(semester.students, settings)
    return make_from_clusters(semester, clusters)

parameters = Parameters(reps = 10)
parameters.preprocess = groups_random
run_repetitions(semester, parameters)

Next, consider when some students form pairs.

In [ ]:
def pairing(semester):
    clusters, _ = make_social_groups_pairs(semester, 0.25, interaction_time = 1200, 
                                           weighted=False)
    return make_from_clusters(semester, clusters)

parameters = Parameters(reps = 10)
parameters.preprocess = pairing
run_repetitions(semester, parameters)

Next, we consider social interactions among varsity athletes.  In the first case, we assume that new social clusters are formed within teams each day.

In [ ]:
def groups_varsity(semester):
        clusters, processed = make_social_groups_varsity(semester, 6, 240, 6, 240)
        return make_from_clusters(semester, clusters)

parameters = Parameters(reps = 10)
parameters.preprocess = groups_varsity
run_repetitions(semester, parameters)

Finally, we consider varsity teams again but assume that the athletes keep socialization within the same cluster of people each day.

In [ ]:
def groups_static_varsity(semester):
        clusters, processed = make_social_groups_varsity_static(semester, 6, 240)
        return make_from_clusters(semester, clusters)

parameters = Parameters(reps = 10)
parameters.preprocess = groups_static_varsity
run_repetitions(semester, parameters)

## Other ways to explore

Here are some other ideas for how to play around with the simulations!

Try changing the infection rate.  Here we retry the first simulation, but with 10% lower infection rate.

In [ ]:
def test_infection_sensitivity(semester, delta):
    parameters = Parameters(reps = 10)
    parameters.rate *= (1 + delta)
    run_repetitions(semester, parameters)

test_infection_sensitivity(semester, -0.10)

Suppose that the percentage of students that are asymptomatic is now 50% instead of 75%.

In [ ]:
def test_asymptomatic(semester, ratio):
    parameters = Parameters(reps = 10)
    parameters.infection_duration = VariedResponse(1 / 3.5, 1 / 4.5, 1 / 2, ratio)
    run_repetitions(semester, parameters)

test_asymptomatic(semester, 0.5)

Eliminate external sources of exposure.

In [ ]:
def test_outsideclass(semester, increase_factor):
    parameters = Parameters(reps = 10)
    parameters.daily_spontaneous_prob *= increase_factor
    run_repetitions(semester, parameters)
    
test_outsideclass(semester, 0)

Change the number of initial infections from 10 to 0 (reasonable if arrival testing is conducted).

In [ ]:
def test_initialconditions(semester, initial_change):
    parameters = Parameters(reps = 10)
    parameters.initial_exposure *= initial_change
    run_repetitions(semester, parameters)
    
test_initialconditions(semester, 0)

Test students once per week, on Sunday.

In [ ]:
def test_test_onceperweek(semester, weekday = 0):
    parameters = Parameters(reps = 10)
    parameters.intervention_policy = IpWeeklyTesting(semester, weekday = weekday)
    run_repetitions(semester, parameters)

test_test_onceperweek(semester, weekday = 6)

Test students one per week, on Monday.

In [ ]:
test_test_onceperweek(semester, weekday = 0)